# 🩺 NLP Médical – Pipeline complet avec transcription + classification + NER + résumé
Ce notebook réalise :
- La transcription d'un audio médical
- Le fine-tuning de CamemBERT sur un jeu de données de classification médicale
- La classification du thème
- L'extraction d'entités médicales (NER)
- Le résumé automatique du texte

**⚠️ Remarque** : ce notebook est prévu pour être exécuté sur Google Colab (GPU recommandé pour l'entraînement).

In [8]:
# 📦 Étape 0 – Installation des dépendances
!pip install -q transformers datasets torchaudio librosa sentencepiece gradio #scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
#!pip install --upgrade transformers
#!pip install scikit-learn
#!pip install numpy pandas --upgrade --force-reinstall
#!pip uninstall pandas numpy -y
#!pip install pandas

In [10]:
# 📁 Étape 1 – Charger le dataset de classification médicale
import pandas as pd
df = pd.read_csv("medical_dataset.csv")
df.head()

,text,label,label_id
0,Le patient souffre d'hypertension et doit surv...,cardio,0
1,On note une tachycardie persistante suite à un...,cardio,0
2,Le patient est très stressé souffre de trouble...,psy,1
3,La dépression est diagnostiquée depuis plusieu...,psy,1
4,La glycémie est instable malgré le traitement ...,diabete,2


In [11]:
import pandas as pd

# Ton jeu de données original
df_orig = pd.read_csv("medical_dataset.csv")

# Données enrichies simulées
data_new = [
    ("Le patient présente une arythmie et des douleurs thoraciques.", "cardio", 0),
    ("Un souffle cardiaque est détecté à l’auscultation.", "cardio", 0),
    ("On observe une bradycardie persistante chez le patient.", "cardio", 0),
    ("Le patient est sujet à des attaques de panique fréquentes.", "psy", 1),
    ("Il rapporte des troubles de concentration et une humeur dépressive.", "psy", 1),
    ("Des signes de schizophrénie ont été détectés au cours de l’entretien.", "psy", 1),
    ("La patiente suit un traitement à l’insuline pour contrôler son diabète.", "diabete", 2),
    ("Une hypoglycémie sévère a été observée après l’effort physique.", "diabete", 2),
    ("Le taux de sucre dans le sang reste élevé malgré le régime adapté.", "diabete", 2),
    ("Le patient souffre d’une crise cardiaque survenue à son domicile.", "cardio", 0),
    ("Il a consulté pour des idées suicidaires persistantes.", "psy", 1),
    ("Un nouveau traitement oral est prescrit pour le diabète de type 1.", "diabete", 2)
]

df_new = pd.DataFrame(data_new, columns=["text", "label", "label_id"])

# Fusionner
df_total = pd.concat([df_orig, df_new], ignore_index=True)
df_total.to_csv("medical_dataset_enriched.csv", index=False)

df_total.to_csv("base_medicale_etendue_.csv", index=False)



In [12]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# === 1. Chargement de ton dataset existant ===
df_total = pd.read_csv("base_medicale_etendue_.csv")  # ou ton fichier actuel

# Mapping labels string vers int
label2id = {"cardio": 0, "psy": 1, "diabete": 2}
df_total['label'] = df_total['label'].map(label2id)

# === 2. Chargement modèle GPT-2 pour génération ===
model_name = "dbddv01/gpt2-french-small"  # Modèle GPT-2 français standard (plus dispo que medical)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# === 3. Fonction de génération ===
def generate_sentences(prompt, n=20, max_length=50):
    sentences = []
    for _ in range(n):
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        outputs = model.generate(
            inputs,
            max_length=max_length,
            do_sample=True,
            temperature=0.9,
            top_p=0.9,
            top_k=50,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # extraire uniquement la génération après le prompt
        generated = text[len(prompt):].strip()
        sentences.append(prompt + generated)
    return sentences

# === 4. Génération pour chaque classe ===
prompts_labels = {
    "cardio": "Le patient présente des symptômes cardiaques tels que",
    "psy": "Le patient souffre de troubles psychologiques comme",
    "diabete": "Le patient est atteint de diabète et présente",
}

generated_data = {"text": [], "label": []}

for label_str, prompt in prompts_labels.items():
    label_id = label2id[label_str]
    gen_sentences = generate_sentences(prompt, n=20)  # 20 phrases par classe
    generated_data["text"].extend(gen_sentences)
    generated_data["label"].extend([label_id]*len(gen_sentences))

# === 5. Créer DataFrame des données générées ===
df_generated = pd.DataFrame(generated_data)

# === 6. Concaténer au dataset initial ===
df_augmented = pd.concat([df_total, df_generated], ignore_index=True)

# === 7. Sauvegarder le dataset augmenté ===
df_augmented.to_csv("base_medicale_augmented.csv", index=False)

print(f"Dataset augmenté : {len(df_augmented)} exemples, dont {len(df_generated)} générés.")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import pandas as pd
from datasets import Dataset

# Charger le fichier CSV
df = pd.read_csv("base_medicale_augmented.csv")  # remplace par le bon nom

# Supprimer la colonne label_id si elle existe
if 'label_id' in df.columns:
    df = df.drop(columns=['label_id'])

# Nettoyer les lignes vides ou incorrectes
df = df.dropna(subset=["text", "label"])
df = df[df["label"].apply(lambda x: str(x).isdigit())]
df["label"] = df["label"].astype(int)

# Vérifier la répartition des classes
print(df["label"].value_counts())
print(df.head())

# Sauvegarde propre (optionnelle)
df.to_csv("base_medicale_propre.csv", index=False)

# Conversion vers Hugging Face Dataset
dataset = Dataset.from_pandas(df)


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# from DATA.MORE_DATA.data_generator import DataGenerator
# data_generator = DataGenerator()
# df = data_generator.get_df_medical()

# 📌 Étape 1 – Chargement et nettoyage
#df = pd.read_csv("base_medicale_propre.csv")
df = pd.read_csv("Dataset_320_sentences.csv")


# Si 'label' est déjà des entiers (0,1,2), on garde uniquement ceux valides
#df = df[df['label'].isin([0, 1, 2])]

# S’assurer que c’est bien un entier
df['label'] = df['label'].astype(int)


# Supprimer les lignes restantes avec NaN
df = df.dropna(subset=["label", "text"])

# Convertir en entier
df['label'] = df['label'].astype(int)


# 💾 Pour debug : vérifier la distribution
print("Label distribution:\n", df['label'].value_counts())

# 📁 Étape 2 – Conversion en Dataset HF
dataset = Dataset.from_pandas(df)

# Split train/test
dataset = dataset.train_test_split(test_size=0.3, seed=42)

# Tokenizer CamemBERT
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

# Préprocessing avec ajout des labels
def preprocess(example):
    tokenized = tokenizer(example['text'], truncation=True, padding='max_length', max_length=128)
    tokenized["labels"] = example["label"]
    return tokenized

# Tokenization
encoded_dataset = dataset.map(preprocess, batched=True)

# Charger modèle CamemBERT pour classification (9 classes)
model = AutoModelForSequenceClassification.from_pretrained("camembert-base", num_labels=9)

# 🎯 Entraînement
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    learning_rate=5e-5,
    report_to="none"
)

# Métriques
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted", zero_division=0)
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


# 🧠 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics
)

# 🚀 Lancer l'entraînement
trainer.train()

# 💾 Sauvegarder modèle et tokenizer
trainer.save_model("./results")
tokenizer.save_pretrained("./results")


In [ ]:
print("Valeurs uniques :", df["label"].unique())
print("Types :", df["label"].apply(type).value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
# Prédictions
preds_output = trainer.predict(encoded_dataset['test'])
y_pred = np.argmax(preds_output.predictions, axis=1)
y_true = preds_output.label_ids

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)
labels = ['cardio', 'psy', 'diabete']

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Prédit')
plt.ylabel('Réel')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
import torch
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Charger modèle et processeur Whisper
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

# Choisir device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Charger audio
audio_path = "diabete_3.mp3"
waveform, sr = torchaudio.load(audio_path)

# Resample à 16kHz
resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
waveform = resampler(waveform)

# Si audio stéréo (2 canaux), moyenne pour mono
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0)

# Convertir en numpy pour WhisperProcessor
waveform_np = waveform.numpy()

# Préparer les input features
inputs = processor(
    waveform_np,
    sampling_rate=16000,
    return_tensors="pt",
    return_attention_mask=False
)

# Padding manuel à 3000 frames (Whisper attend cette taille exacte)
input_features = inputs["input_features"]
if input_features.shape[-1] < 3000:
    pad_len = 3000 - input_features.shape[-1]
    input_features = torch.nn.functional.pad(input_features, (0, pad_len), mode="constant", value=0)

inputs["input_features"] = input_features.to(device)

# Générer la transcription
with torch.no_grad():
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="fr", task="transcribe")
    generated_ids = model.generate(
        inputs["input_features"],
        forced_decoder_ids=forced_decoder_ids
    )

# Décoder la sortie en texte
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("📄 Transcription :", transcription)
print("🖥️ Device utilisé :", device)


In [ ]:
# 🩺 Étape 4 – Classification du thème médical avec le modèle entraîné
from transformers import pipeline, AutoModelForSequenceClassification

model_cls = AutoModelForSequenceClassification.from_pretrained("./results")
classifier = pipeline("text-classification", model=model_cls, tokenizer=tokenizer, return_all_scores=True)

# Appliquer la classification sur la transcription
all_scores = classifier(transcription)

# Affichage des scores pour chaque classe
print("🩺 Scores des thèmes médicaux détectés :")
for score in all_scores[0]:
    print(f"{score['label']}: {round(score['score']*100, 2)} %")


In [ ]:
#!pip install huggingface_hub --upgrade

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Thibeb/DrBert_generalized")
model = AutoModelForTokenClassification.from_pretrained("Thibeb/DrBert_generalized")

ner_pipelineTh = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# 📄 Charger ta base (fichier CSV ou dataframe si déjà en mémoire)
df = pd.read_csv("medical_dataset_enriched.csv")  # ou ton fichier local

# 🔍 Appliquer le NER à chaque ligne du texte
def extract_entities(text):
    entities = ner_pipelineTh(text)
    return [(ent['word'], ent['entity_group']) for ent in entities]

df["entities"] = df["text"].apply(extract_entities)

# 🔎 Afficher quelques résultats
print(df[["text", "entities"]].head(10))

# 💾 Optionnel : sauvegarde dans un nouveau fichier CSV
df.to_csv("medical_dataset_with_entities_.csv", index=False)


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

transcription2 = "Bonjour, le patient Mr Dupond présente une douleur et un diabète de type 2 non contrôlé avec une glycémie élevée et prend de la metformine."

models = {
    "DrBert_generalized": "Thibeb/DrBert_generalized",
    "Camembert_NER": "Jean-Baptiste/camembert-ner",
    "Thibeb/DrBert": "Thibeb/DrBert_generalized"
}

for name, model_path in models.items():
    print(f"\n--- Résultats avec {name} ---")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    entities = ner_pipeline(transcription)
    if entities:
        for ent in entities:
           print(f"Entité: {ent['word']}, Type: {ent['entity_group']}, Score: {ent['score']:.3f}")
    else:
        print("Aucune entité détectée")


In [ ]:
# ✂️ Étape 6 – Résumé automatique du texte
summarizer = pipeline("summarization", model="plguillou/t5-base-fr-sum-cnndm")
resume = summarizer(transcription, max_length=100, min_length=20, do_sample=False)[0]['summary_text']
print("📝 Résumé :", resume)